In [ ]:
import numpy as np
import pandas as pd
import pamillia_analyis
import cv2

import re
import os.path
import os
from pathlib import Path
import glob
import tqdm
import tqdm.notebook
import multiprocessing


import skimage
import skimage.io
import skimage.util
import skimage.morphology
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale


In [ ]:
metadata = pd.read_excel("C:/Users/jwagne/Desktop/pamillia_spme_metadata_v2.xlsx", sheet_name='Sheet1')

DLC_prefix = "C:/Users/jwagne/git/pamillia_only-julian_wagner-2020-08-05/videos/"
DLC_postfix = "DLC_resnet50_pamillia_onlyAug5shuffle1_150000.h5"
dlc_files_1 = [re.sub('/volume1/Parker_lab/julian/[^/]+/', DLC_prefix, re.sub('.fmf', DLC_postfix, metadata.iloc[i]['video_file'])) for i in range(len(metadata))]

DLC_postfix = "DLC_resnet50_pamillia_onlyAug5shuffle1_250000.h5"
dlc_files_2 = [re.sub('/volume1/Parker_lab/julian/[^/]+/', DLC_prefix, re.sub('.fmf', DLC_postfix, metadata.iloc[i]['video_file'])) for i in range(len(metadata))]
metadata['DLC_file'] = [*dlc_files_1[0:16], *dlc_files_2[16:]]

vid_prefix = "C:/Users/jwagne/git/pamillia_only-julian_wagner-2020-08-05/videos/"
vid_postfix = ".mp4"
local_vid_files = [re.sub('/volume1/Parker_lab/julian/[^/]+/', vid_prefix, re.sub('.fmf', vid_postfix, metadata.iloc[i]['video_file'])) for i in range(len(metadata))]
metadata['video_file_local'] = local_vid_files

metadata['timestamp_file'] = [*[re.sub('.mp4', '_timestamps.csv', metadata.iloc[i]['video_file_local']) for i in range(len(metadata))][0:16],
                              *[re.sub('.mp4', '.csv', metadata.iloc[i]['video_file_local']) for i in range(len(metadata))][16:]]

x1_crops = [160,  150,  150,  150,  150,  140,  145,  155,  155,  185,  185,  180,  180,  185,  180,  185,
            160,  160,  160,  155,  160,  155,  150,  165,  160,  160,  160,  165,  165,  160,  160,  165]
x2_crops = [1230, 1230, 1225, 1225, 1240, 1235, 1235, 1235, 1240, 1280, 1280, 1280, 1275, 1280, 1280, 1280,
            1250, 1255, 1255, 1245, 1250, 1245, 1245, 1260, 1260, 1260, 1260, 1260, 1260, 1260, 1255, 1260]

metadata['x1_crop'] = x1_crops
metadata['x2_crop'] = x2_crops

metadata['pixPerMm'] = (metadata['x2_crop']-metadata['x1_crop'])/35

print(metadata['DLC_file'][0])
metadata.head()

If the bug is using its gland for defense, another behavioral readout that may elicit gland usage are close encounters with ants. Hence, the next analysis approach will quantify how close the bug is to an ant at all times in the arena, as well as extract the amount of time the bug was 'close' to the ant based on a threshold. The algorithmic approach here goes like this:
1) Use [DeepLabCut](https://github.com/DeepLabCut/DeepLabCut) (DLC) to locate the position of the bug, and label points along its body  
2) Background subtract from the behavioral video to get clean capture of animal positions  
3) Remove the bug from the frame using the DLC positions  
4) Use thresholding to locate ant positions as blobs  
5) Find smallest distance from the ant positions to the bug with blob positions and DLC data


In order to do this, we first construct background models for the videos by applying a median filter to the several frames in the videos with large amounts of time between frames. Since the animals are moving around the arena, if we use a median filter and take the dark pixels from an array of different frames (these videos are top lit against a dark background), this will eliminate the animals and leave the darker background pixel values. We generate the background images here.

In [ ]:
bg_ims = []
bg_im_dict = {}
#cap.release()
buffer_spme = 20
for ind in tqdm.notebook.tqdm(range(len(metadata))):
    
    if ';' in metadata.iloc[ind]['video_file']:
        file1, file2 = re.match('([^;]+);([^;]+)', metadata.iloc[ind]['video_file']).groups()
        for il, file in enumerate([file1, file2]):
            file = re.sub('.fmf', '.mp4', file)
            file = re.sub('/volume1/Parker_lab', 'Z:', file)
            
            start_time=0
            if il == 0:
                start_time = metadata.iloc[ind]['SPME_start_time'] + buffer_spme
            stop_time = metadata.iloc[ind]['SPME_stop_time'] - buffer_spme
            frame_rate = 25

            cap = cv2.VideoCapture(file)
            frames = []
            for inds in np.linspace(start_time*frame_rate, cap.get(7)-1, 30).astype(int):
                cap.set(1, inds)
                ret, frame = cap.read()
                frames.append(skimage.img_as_float32(frame[:, :, 0]))
            bg_ims.append((np.percentile(np.array(frames), q=0.1, axis=0), file))
            bg_im_dict[metadata.iloc[ind]['video_file']] = ind
            break

    else:
        raw_file_name = metadata.iloc[ind]['video_file']
        file = re.sub('.fmf', '.mp4', metadata.iloc[ind]['video_file'])
        file = re.sub('/volume1/Parker_lab', 'Z:', file)

        start_time = metadata.iloc[ind]['SPME_start_time'] + buffer_spme
        stop_time = metadata.iloc[ind]['SPME_stop_time'] - buffer_spme
        frame_rate = 25
        
        cap = cv2.VideoCapture(file)
        frames = []
        for inds in np.linspace(start_time*frame_rate, stop_time*frame_rate, 30).astype(int):
            cap.set(1, inds)
            ret, frame = cap.read()
            frames.append(skimage.img_as_float32(frame[:, :, 0]))
        bg_ims.append((np.percentile(np.array(frames), q=0.1, axis=0), file))
        bg_im_dict[raw_file_name] = ind

Prep args for multiproccessing on all vids.

In [ ]:
args = []
include_bools = (metadata['interactor'] == '5_locc') | (metadata['interactor'] == '5_locc_glued')
for ind in range(len(metadata)):
    if not include_bools[ind]:
        continue
    
    if ';' in metadata.iloc[ind]['video_file']:
        continue
        
    vid_name = re.sub('.mp4', '_blob_dist_an_vid.mp4', metadata.iloc[ind]['video_file_local'])
    x1_crop = metadata.iloc[ind]['x1_crop']
    x2_crop = metadata.iloc[ind]['x2_crop']
    start_ind = int(metadata.iloc[ind]['SPME_start_time']*25)
    stop_ind = int(metadata.iloc[ind]['SPME_stop_time']*25)
    ine = np.linspace(start_ind, stop_ind, (stop_ind-start_ind+1)).astype(int)#[31529:31533]
    
    args.append((metadata, 
                 bg_ims,
                 bg_im_dict,
                 0.05, #thresh
                 ind, #ind
                 ine, #ine
                 1000, #min_size
                 x1_crop, #x1_crop
                 x2_crop, #x2_crop
                 (20, 30, 30, 30, 20), #ball_rads
                 20, #buffer_spme
                 25, #frame_rate
                 ['bh', 'bt1', 'bt2', 'ba1', 'ba2'], #parts
                 0.5, #likelihood_thresh
                 True, #give_ims
                 True, #save_vid
                 vid_name, #vid_name
                 'DLC_resnet50_pamillia_onlyAug5shuffle1_150000', #network
                 10, #erode_amount
                 10, #dilate_amount
                 False, #return_ims
                 100, #part_shift_thresh
                 'video_file_local' #vid_col
                ))

Run analysis of distance to blobs on all vids.

In [ ]:
def multi_proc_pamillia_dists(arguments, proc_num=14):
    if __name__ == '__main__':
        with multiprocessing.Pool(processes = proc_num) as pool:
            result = pool.starmap(pamillia_analyis.ant_blobs_dists, arguments)
    return(result)